<a href="https://colab.research.google.com/github/kayneh-dev/IAT360-FinalProject-RAGChatbot/blob/main/Another_copy_of_IAT360_FinalProject_RAGChatbot_CRAWL4AI_verReducedMem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [1]:
!pip install crawl4ai sentence-transformers faiss-cpu transformers accelerate torch bitsandbytes --quiet
!pip install -U crawl4ai
!pip install nest_asyncio


In [2]:
# Check crawl4ai version
import crawl4ai
print(crawl4ai.__version__.__version__)

0.7.7


In [3]:
%%capture
!crawl4ai-setup

In [4]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.7.7 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 1.09s 
[FETCH]... ↓ https://crawl4ai.com                                               
| ✓ | ⏱: 3.96s 
[SCRAPE].. ◆ https://crawl4ai.com                                               
| ✓ | ⏱: 0.10s 
[COMPLETE] ● https://crawl4ai.com                                               
| ✓ | ⏱: 4.07s 
[COMPLETE] ● ✅ Crawling test passed! 


Testing if PlayWright is installed

In [5]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
import asyncio
from playwright.async_api import async_playwright

async def test_browser():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto('https://example.com')
        print(f'Title: {await page.title()}')
        await browser.close()

asyncio.run(test_browser())

Title: Example Domain


In [7]:
!pip install transformers torch accelerate

Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.



In [8]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `IAT360-Final-TestRead` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /

In [9]:
!huggingface-cli whoami

⚠️  Warning: 'huggingface-cli whoami' is deprecated. Use 'hf auth whoami' instead.
kayneh


Define SFU URLs

In [10]:
sfu_urls = [
    "https://www.sfu.ca/students/deadlines/spring.html",
    "https://www.sfu.ca/students/deadlines/summer.html",
    "https://www.sfu.ca/students/deadlines/fall.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/major/bachelor-of-arts.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/major/bachelor-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/honours/bachelor-of-arts.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/honours/bachelor-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/communication-and-interactive-arts-and-technology/joint-major/bachelor-of-arts.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/communication-and-interactive-arts-and-technology/joint-major/bachelor-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology-and-business/joint-major/bachelor-of-arts-or-bachelor-of-business-administration.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology-and-business/joint-major/bachelor-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/minor.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/digital-journalism/certificate.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/sound/certificate.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/co-operative-education-program.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology-second-degree/bachelor-of-arts.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology-second-degree/bachelor-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/doctor-of-philosophy.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/master-of-arts.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/interactive-arts-and-technology/master-of-science.html",
    "https://www.sfu.ca/students/calendar/2026/spring/programs/visual-analytics/graduate-certificate.html",
    "https://www.sfu.ca/siat/people.html",
    "https://www.sfu.ca/siat/people/staff.html",
    "https://www.sfu.ca/siat/people/research-faculty.html",
    "https://www.sfu.ca/siat/people/teaching-faculty.html",
    "https://www.sfu.ca/siat/people/emeritus-faculty.html",
    "https://www.sfu.ca/siat/people/adjunct-faculty.html",
    "https://www.sfu.ca/siat/people/alumni.html",
    "https://www.sfu.ca/siat/people/work-at-siat.html",
    "https://www.sfu.ca/siat/people/opportunities.html",
    "https://www.sfu.ca/coop/programs/siat.html",
    "https://www.sfu.ca/coop/programs/siat/prospective.html",
    "https://www.sfu.ca/coop/programs/siat/new.html",
    "https://www.sfu.ca/coop/programs/siat/seeking.html",
    "https://www.sfu.ca/coop/programs/siat/working.html",
    "https://www.sfu.ca/coop/programs/siat/sample-jobs.html",
    "https://www.sfu.ca/coop/programs/siat/contact.html",
    "https://www.sfu.ca/coop/apply.html",
    "https://www.sfu.ca/coop/costs-and-deadlines.html",
    "https://www.sfu.ca/coop/roadmap.html"
]

Crawl SFU pages with crawl4ai → get markdown

In [11]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

async def crawl_sfu_pages(urls):
    browser_config = BrowserConfig(
        headless=True,
        verbose=True,
        # Stop Chrome/Playwright from touching the GPU
        extra_args=[
            "--disable-gpu",
            "--disable-software-rasterizer",
            "--disable-dev-shm-usage",
        ],
    )

    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,  # always get fresh content
        stream=False                  # get all results at once
    )

    # Run many URLs in parallel using Crawl4AI's internal dispatcher
    async with AsyncWebCrawler(config=browser_config) as crawler:
        crawl_results = await crawler.arun_many(
            urls=urls,
            config=run_config
        )

    # Convert CrawlResult objects into your existing raw_docs format
    results = []
    for result in crawl_results:
        if result.success:
            results.append({
                "url": result.url,
                "markdown": result.markdown,   # ready for RAG
            })
        else:
            print("Failed:", result.url, "->", result.error_message)

    return results

# Run the crawl (Colab supports asyncio.run)
raw_docs = asyncio.run(crawl_sfu_pages(sfu_urls))

len(raw_docs), [d["url"] for d in raw_docs]



[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://www.sfu.ca/coop/programs/siat.html                                                           |
✓ | ⏱: 14.71s 

[SCRAPE].. ◆ https://www.sfu.ca/coop/programs/siat.html                                                           |
✓ | ⏱: 1.05s 

[COMPLETE] ● https://www.sfu.ca/coop/programs/siat.html                                                           |
✓ | ⏱: 15.89s 

[FETCH]... ↓ https://www.sfu.ca/coop/apply.html                                                                   |
✓ | ⏱: 30.59s 

...

[COMPLETE] ● https://www.sfu.ca/students/deadlines/summer.html                                                    |
✓ | ⏱: 35.41s 

[FETCH]... ↓ https://www.sfu.ca/students/calendar/2026/spring...-arts-and-technology/major/bachelor-of-arts.html  |
✓ | ⏱: 46.36s 

[SCRAPE].. ◆ https://www.sfu.ca/students/calendar/2026/spring...-arts-and-technology/major/bachelor-of-arts.html  |
✓ | ⏱: 0.26s 

[COMPLETE] ● https://www.sfu.ca/students/calendar/2026/spring...-arts-and-technology/major/bachelor-of-arts.html  |
✓ | ⏱: 46.63s 

(40,
 ['https://www.sfu.ca/coop/programs/siat.html',
  'https://www.sfu.ca/coop/apply.html',
  'https://www.sfu.ca/coop/programs/siat/new.html',
  'https://www.sfu.ca/coop/programs/siat/sample-jobs.html',
  'https://www.sfu.ca/siat/people/emeritus-faculty.html',
  'https://www.sfu.ca/siat/people/teaching-faculty.html',
  'https://www.sfu.ca/coop/programs/siat/contact.html',
  'https://www.sfu.ca/siat/people/alumni.html',
  'https://www.sfu.ca/siat/people/research-faculty.html',
  'https://www.sfu.ca/siat/people.html',
  'https://www.sfu.ca/coop/programs/siat/prospective.html',
  'https://www.sfu.ca/coop/roadmap.html',
  'https://www.sfu.ca/coop/programs/siat/seeking.html',
  'https://www.sfu.ca/siat/people/work-at-siat.html',
  'https://www.sfu.ca/siat/people/adjunct-faculty.html',
  'https://www.sfu.ca/coop/programs/siat/working.html',
  'https://www.sfu.ca/siat/people/opportunities.html',
  'https://www.sfu.ca/siat/people/staff.html',
  'https://www.sfu.ca/coop/costs-and-deadlines.ht

In [12]:
print(raw_docs[0])

{'url': 'https://www.sfu.ca/coop/programs/siat.html', 'markdown': '  * [sfu.ca](https://www.sfu.ca/)\n\n\n[instagram](https://www.instagram.com/sfu_coop) [linkedin](http://www.linkedin.com/groups/4116586/)\n  * [SIGN IN TO MYEXPERIENCE ↗](https://www.sfu.ca/coop/myexperience.html "SIGN IN TO MYEXPERIENCE ↗")\n  * [HIRE OUR STUDENTS ↗](https://www.sfu.ca/coop/hire.html "HIRE OUR STUDENTS ↗")\n  * [CO-OP STAFF RESOURCES](https://www.sfu.ca/coop/wil-staff.html "CO-OP STAFF RESOURCES")\n\n\n![](https://www.sfu.ca/etc/designs/clf/clientlibs/clf4/default/img/sfu-logo-only@2x.png)\nMENU undefined undefined\n[ ![](https://www.sfu.ca/etc/designs/clf/clientlibs/clf4/default/img/SFU@2x.png) ![](https://www.sfu.ca/coop/programs/siat.html) ![](https://www.sfu.ca/coop/programs/siat.html) ](https://www.sfu.ca/coop.html)\n[ Co-operative Education ](https://www.sfu.ca/coop.html)\n  * sign in\n    * [SFU Mail](https://outlook.office.com)\n    * [goSFU](https://go.sfu.ca)\n    * [Canvas](https://canvas.s

Chunk the markdown text

In [13]:
import re
from typing import List, Dict

def split_by_headings(markdown: str) -> List[str]:
    parts = re.split(r"(?=^#{1,6}\s)", markdown, flags=re.MULTILINE)
    return [p.strip() for p in parts if p.strip()]

def chunk_text(text: str, max_chars: int = 1500) -> List[str]:
    """
    Chunk text by sentences so we don't cut in the middle of sentences or words.
    """
    # Simple sentence split: split after ., ?, ! followed by whitespace
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ""

    for sent in sentences:
        # If adding this sentence would exceed the limit, start a new chunk
        if len(current) + len(sent) + 1 > max_chars:
            if current.strip():
                chunks.append(current.strip())
            current = sent
        else:
            current = (current + " " + sent).strip()

    if current.strip():
        chunks.append(current.strip())

    return chunks

chunks: List[Dict] = []

for doc in raw_docs:
    sections = split_by_headings(doc["markdown"])
    for sec in sections:
        for c in chunk_text(sec, max_chars=1500):
            chunks.append({
                "url": doc["url"],
                "text": c,
            })

len(chunks)



1432

Create embeddings + FAISS index

In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. Load embedding model
embed_model = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2",
    device="cpu"  # keep embeddings on CPU so GPU is reserved for Llama 2
)


# 2. Embed all chunks
corpus_texts = [c["text"] for c in chunks]
corpus_embeddings = embed_model.encode(
    corpus_texts,
    convert_to_numpy=True,
    show_progress_bar=True
)

# 3. Normalize for cosine similarity
corpus_norms = np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
corpus_embeddings = corpus_embeddings / corpus_norms

# 4. Build FAISS index
dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(corpus_embeddings)

print("Index size:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Index size: 1432


retrieval helper

In [15]:
def retrieve_chunks(query: str, k: int = 5):
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)
    scores, indices = index.search(q_emb, k)

    results = []
    for rank, (idx, score) in enumerate(zip(indices[0], scores[0]), start=1):
        if idx == -1:
            continue
        c = chunks[idx]
        results.append({
            "rank": rank,
            "score": float(score),
            "chunk_index": int(idx),
            "text": c["text"],
            "url": c["url"],
        })
    return results

# quick test
for r in retrieve_chunks("Withdrawal deadline for Spring 2026", k=3):
    print("RANK", r["rank"], "SCORE", r["score"], "URL", r["url"])
    print(r["text"][:250], "...\n")
    print("-" * 80)


RANK 1 SCORE 0.4835790991783142 URL https://www.sfu.ca/students/deadlines/summer.html
If withdrawing from all classes, a withdrawal notation of “Withdrew YYYY-MM-DD” will be added to your academic record. Sunday, July 6  
| Last day to drop a class using [goSFU](http://go.sfu.ca/), with a withdrawal notation (WD) on your academic reco ...

--------------------------------------------------------------------------------
RANK 2 SCORE 0.4691503643989563 URL https://www.sfu.ca/students/deadlines/summer.html
---|---  
Sunday, July 6  
| Last day to drop a summer session class on [goSFU](http://go.sfu.ca/), without a withdrawal notation of “WD” for the course on your academic record. If withdrawing from all classes, a withdrawal notation of “Withdrew YYYY ...

--------------------------------------------------------------------------------
RANK 3 SCORE 0.4690212905406952 URL https://www.sfu.ca/students/deadlines/spring.html
If withdrawing from all classes, a withdrawal notation of “Withdrew 

Load Llama

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"

# 4-bit quantization config (NF4) to reduce VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",          # NormalFloat4 for good accuracy
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",                  # Let HF place layers across GPU/CPU
    use_auth_token=True,
)

print(f"Model loaded. Approx VRAM: {model.get_memory_footprint() / 1e9:.2f} GB")


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded. Approx VRAM: 3.76 GB


In [17]:
from transformers import pipeline

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # The quantized model already knows its dtype/device; no need to override
    max_new_tokens=256,
    do_sample=False,
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [18]:
SYSTEM_PROMPT = """You are an SFU academic assistant.
You ONLY answer questions using the SFU content provided in CONTEXT.
If the answer is not in the context, say you do not know and suggest contacting SFU advisors.
Always:
- Answer in plain, student-friendly English.
- Do NOT invent deadlines, GPA thresholds, or policy rules.
- Do NOT include URLs yourself; the system will add a Sources section with URLs.
"""

import re

def remove_urls(text: str) -> str:
    # Remove http/https and www links
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    # Also remove bare "sfu.ca/..." style links
    text = re.sub(r"\b\S*\.sfu\.ca\S*", "", text)
    # Clean up extra spaces left behind
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def build_rag_prompt(question: str, retrieved: list) -> str:
    context_parts = []
    for r in retrieved:
        context_parts.append(
            f"[{r['rank']}] Source: {r['url']}\n{r['text']}"
        )
    context_block = "\n\n---\n\n".join(context_parts)

    prompt = f"""{SYSTEM_PROMPT}

CONTEXT:
{context_block}

QUESTION:
{question}

INSTRUCTIONS:
- Use only the information in CONTEXT.
- Quote key sentences when helpful.
- Do NOT write any URLs or links in your answer. The system will add URLs in a separate Sources section.
- If the context is conflicting or unclear, say so.

ANSWER:"""
    return prompt

def get_llm_response(question: str, top_k: int = 5):
    retrieved = retrieve_chunks(question, k=top_k)
    if not retrieved:
        print("Chatbot: I couldn't find any relevant SFU information in my documents. Please check the SFU website or contact an advisor.")
        return

    prompt = build_rag_prompt(question, retrieved)

    outputs = llm(
        prompt,
        do_sample=False,
        max_new_tokens=256,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    full_text = outputs[0]["generated_text"]
    answer = full_text.split("ANSWER:", 1)[-1].strip() if "ANSWER:" in full_text else full_text

    #–––––––––––––––––––––––––––––––––––––––––––––
    # SELECT ONLY THE TOP 1–2 UNIQUE URLs
    # (based on retrieval ranking)
    #–––––––––––––––––––––––––––––––––––––––––––––
    seen = set()
    ranked_urls = []

    for r in retrieved:
        url = r["url"]
        if url not in seen:
            ranked_urls.append(url)
            seen.add(url)

    # Limit to only the top 1 or 2 URLs
    final_urls = ranked_urls[:2]  # <-- adjust to [:1] if you want exactly 1

    sources_block = "\n\nSources:\n" + "\n".join(f"- {u}" for u in final_urls)

    print("Chatbot:", answer + sources_block)


In [19]:
#Testing llm response
get_llm_response("I am a new SFU student. How do I declare my major?")


Chatbot: To declare your major at SFU, you need to follow these steps:

1. Go to [myExperience](https://cas.sfu.ca/cas/login?message=Welcome+to+SFU+myExperience.%20Please+login+with+your+SFU+computing+ID.&allow=student,alumni&renew=true&service=https://myexperience.sfu.ca/sfuLogin.htm%3Faction%3Dlogin) and sign in as a student using your SFU ID.
2. Click on the "Exp. Learning (Co-op)" option in the left navigation bar.
3. Click on "Apply to a Program" near the upper right corner.
4. Select the Co-op Program that corresponds with your faculty program.
5. Complete the online application form and agreement, and click "Submit."

Note: The admission requirements for your major can be found in the SFU Calendar, specifically in the program pages for your faculty. You can access the Calendar through the SFU website.

---

Sources:

[1]

Sources:
- https://www.sfu.ca/coop/roadmap.html
- https://www.sfu.ca/coop/apply.html


###Our Official RAG chatbot

In [28]:
def is_on_topic(q: str) -> bool:
    q = q.lower()
    keywords = ["sfu", "siat", "course", "enrol", "enroll", "calendar", "co-op", "coop", "gpa", "academic", "credits", "withdrawal", "refund", "fee", "payment", "deadline", "prerequisite", "class", "fall", "spring", "summer"]
    return any(k in q for k in keywords)

print("SFU RAG Chatbot with crawl4ai. Type 'exit' to quit.\n")

while True:
    user_q = input("You: ")
    if user_q.lower().strip() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")


        break

    if not is_on_topic(user_q):
        print("Chatbot: I can only answer questions about SFU academic information.")
        continue

    get_llm_response(user_q)


SFU RAG Chatbot with crawl4ai. Type 'exit' to quit.



KeyboardInterrupt: Interrupted by user

###Testing the performance of the chatbot
Performance metrics

In [21]:
def generate_answer_and_sources(question: str, top_k: int = 5):
    """
    Run the full RAG chatbot for a question:
    - retrieve chunks
    - build RAG prompt (including SYSTEM_PROMPT)
    - call llm(...)
    Returns: answer_text, final_urls, retrieved_chunks
    """
    retrieved = retrieve_chunks(question, k=top_k)
    if not retrieved:
        return None, [], []

    prompt = build_rag_prompt(question, retrieved)

    outputs = llm(
        prompt,
        do_sample=False,        # deterministic for evaluation
        max_new_tokens=256,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    full_text = outputs[0]["generated_text"]
    answer = full_text.split("ANSWER:", 1)[-1].strip() if "ANSWER:" in full_text else full_text
    answer = remove_urls(answer)


    # Take top 1–2 unique URLs from retrieved chunks
    seen = set()
    ranked_urls = []
    for r in retrieved:
        url = r["url"]
        if url not in seen:
            ranked_urls.append(url)
            seen.add(url)

    final_urls = ranked_urls[:2]

    return answer, final_urls, retrieved


In [30]:
labeled_chatbot_data = [
    {
        "id": 1,
        "question": "What is the course withdrawal deadline for the Spring term?",
        "relevant_urls": [
            "https://www.sfu.ca/students/deadlines/spring.html"
        ],
        "gold_answer": "Spring term courses start January 5, 2026; last day of exams is April 25, 2026.**    November 3, 2025: enrolment appointments for Spring 2026 classes begin   November 24, 2025: open enrolment period begins at 8:00 am. The Spring course withdrawal deadline is listed on the SFU Spring deadlines page. Students must drop courses by that official deadline for the withdrawal to be recorded properly."
    },
    {
        "id": 2,
        "question": "What is SFU co-op and what is it for?",
        "relevant_urls": [
            "https://www.sfu.ca/coop/advice-and-support.html",
            "https://www.sfu.ca/coop/programs/siat.html"
        ],
        "gold_answer": "SFU Co-op allows students to alternate between academic study terms and paid work terms. It helps students gain real work experience, develop professional skills, and build industry connections."
    },

    {
        "id": 3,
        "question": "How can I contact an academic advisor about planning my courses?",
        "relevant_urls": [
            "https://www.sfu.ca/students/academicadvising/contact.html",
            "https://www.sfu.ca/siat/programs/undergraduate/current-students/advising-and-support.html"
        ],
        "gold_answer": (
            "To contact an academic advisor about planning your courses, you can visit the SFU Advising website at [1]. On this website, you can find the contact information for the academic advisors in your program, as well as the hours of operation and appointment availability. You can also email the advisors directly or schedule an appointment through the online booking system. "
            "They offer drop-in advising, LiveChat, and booked one-on-one appointments. "
            "SIAT students can also use the SIAT advising page for program-specific support."
        )
    },
    {
        "id": 4,
        "question": "How much are undergraduate tuition fees per term at SFU?",
        "relevant_urls": [
            "https://www.sfu.ca/students/calendar/fees-and-regulations/tuition-fees/undergraduate.html",
            "https://www.sfu.ca/students/admission/fees-scholarships.html",
            "https://www.sfu.ca/students/financial-aid/plan/typical-costs.html"
        ],
        "gold_answer": (
            "Undergraduate tuition at SFU is charged mainly by the number of units you take, "
            "and the exact amount depends on your course load and student category. "
            "You should use the official tuition fee and cost calculator pages to see current estimates."
        )
    },
    {
        "id": 5,
        "question": "What GPA do I need to transfer into SFU from another Canadian university?",
        "relevant_urls": [
            "https://www.sfu.ca/students/admission/admission-requirements/canadian-transfer/college-university.html",
            "https://www.sfu.ca/students/admission/admission-requirements.html"
        ],
        "gold_answer": (
            "For most Canadian university transfer applicants, SFU requires you to be in good standing "
            "with at least a 2.00 GPA, but admission is competitive. "
            "Some programs may need higher grades than the minimum."
        )
    },
    {
        "id": 6,
        "question": "When should I apply for SIAT co-op?",
        "relevant_urls": [
            "https://www.sfu.ca/coop/programs/siat/prospective.html",
            "https://www.sfu.ca/siat/programs/undergraduate/current-students/co-op.html"
        ],
        "gold_answer": (
            "SIAT students are encouraged to apply for co-op about two terms before their desired work term, "
            "ideally when they have roughly 30 to 80 credit hours. "
            "Applying early gives you time to prepare and be approved before job searches begin."
        )
    },
    {
        "id": 7,
        "question": "Where can I find typical cost estimates for studying at SFU, including living expenses?",
        "relevant_urls": [
            "https://www.sfu.ca/students/financial-aid/plan/typical-costs.html",
            "https://www.sfu.ca/students/admission/fees-scholarships.html"
        ],
        "gold_answer": (
            "SFU’s Financial Aid and Awards site has a Typical Costs page that shows sample budgets for tuition, "
            "fees, housing, food, and books. You can also use the cost calculator on the admission site "
            "to estimate your term costs."
        )
    },
    {
        "id": 8,
        "question": "How can I book an online appointment with an academic advisor?",
        "relevant_urls": [
            "https://www.sfu.ca/students/academicadvising/advising-hours.html",
            "https://www.sfu.ca/students/academic-success.html"
        ],
        "gold_answer": (
            "To book an online appointment, you normally sign in to Advisor Link from the Academic Advising pages. "
            "You can also use drop-in or LiveChat hours for quick questions, but longer planning usually needs a booked appointment."
        )
    },

    # NO-ANSWER examples – your system *should* say it does not know
    {
        "id": 9,
        "question": "What is the weather like on Burnaby Mountain right now?",
        "relevant_urls": [],
        "gold_answer": ""
    },
    {
        "id": 10,
        "question": "Which SFU professor is the easiest marker for SIAT courses?",
        "relevant_urls": [],
        "gold_answer": ""
    },
]


In [23]:
import re

def normalize_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def answer_similarity(predicted: str, gold: str) -> float:
    """
    Very simple token overlap score between 0 and 1.
    1.0 means the predicted answer covers all gold tokens.
    """
    p_norm = normalize_text(predicted)
    g_norm = normalize_text(gold)

    p_tokens = set(p_norm.split())
    g_tokens = set(g_norm.split())

    if not g_tokens:
        return 0.0

    overlap = p_tokens.intersection(g_tokens)
    return len(overlap) / len(g_tokens)


In [24]:
def evaluate_full_chatbot(dataset, top_k: int = 5, name: str = "Chatbot"):
    """
    Evaluate the full RAG chatbot (retrieval + system prompt + generation).

    For each example:
    - Runs generate_answer_and_sources (full pipeline)
    - Checks if any returned URL is relevant (Hit@k)
    - Computes answer similarity vs gold reference answer
    - Prints per-example details and summary metrics
    """
    if not dataset:
        print(f"{name}: dataset is empty.")
        return

    retrieval_hits = []
    similarities = []

    for case in dataset:
        q = case["question"]
        relevant_urls = case["relevant_urls"]
        gold_answer = case["gold_answer"]

        print(f"\n=== [{name}] Example {case['id']} ===")
        print("Question:", q)

        answer, urls, retrieved = generate_answer_and_sources(q, top_k=top_k)

        if answer is None:
            print("No answer (no retrieved chunks).")
            retrieval_hits.append(0.0)
            similarities.append(0.0)
            continue

        print("\nModel answer (truncated):")
        print(answer[:400] + ("..." if len(answer) > 400 else ""))

        print("\nReturned URLs:")
        for u in urls:
            print(" -", u)

        # Retrieval metric: did we get any relevant URL?
        retrieval_hit = any(
            any(rel in u for rel in relevant_urls) for u in urls
        )
        retrieval_hits.append(1.0 if retrieval_hit else 0.0)

        # Answer similarity metric
        sim = answer_similarity(answer, gold_answer)
        similarities.append(sim)

        print("\nRetrieval hit (any URL relevant):", retrieval_hit)
        print(f"Answer similarity vs gold (0–1): {sim:.2f}")

        # OPTIONAL: also show top retrieved chunks
        print("\nTop retrieved chunks (debug):")
        for r in retrieved:
            print(f"[Rank {r['rank']}] {r['url']} (score={r['score']:.3f})")
            preview = r["text"][:250].replace("\n", " ")
            if len(r["text"]) > 250:
                preview += " ..."
            print("  ", preview)
            print()

    # Summary
    n = len(dataset)
    mean_hit = sum(retrieval_hits) / n
    mean_sim = sum(similarities) / n

    print(f"\n=== {name} Summary over {n} questions ===")
    print(f"Retrieval Hit@{top_k}: {mean_hit:.2f}")
    print(f"Average answer similarity: {mean_sim:.2f}")


In [31]:
evaluate_full_chatbot(labeled_chatbot_data, top_k=5, name="SFU RAG Chatbot")



=== [SFU RAG Chatbot] Example 1 ===
Question: What is the course withdrawal deadline for the Spring term?

Model answer (truncated):
The course withdrawal deadline for the Spring term is January 25, 2026. According to the provided context, this is the last day to add, swap, or change tutorials for the Spring term, and it is also the last day to enrol or change to audit by the department offering the course.

Returned URLs:
 - https://www.sfu.ca/students/deadlines/spring.html
 - https://www.sfu.ca/students/deadlines/summer.html

Retrieval hit (any URL relevant): True
Answer similarity vs gold (0–1): 0.31

Top retrieved chunks (debug):
[Rank 1] https://www.sfu.ca/students/deadlines/spring.html (score=0.650)
   ## Spring 2026    **Spring term courses start January 5, 2026; last day of exams is April 25, 2026.**    November 3, 2025: enrolment appointments for Spring 2026 classes begin   November 24, 2025: open enrolment period begins at 8:00 am **Note:** For ...

[Rank 2] https://www.sfu.

In [26]:
from sklearn.metrics import confusion_matrix

def evaluate_chatbot_confusion_matrix(dataset, top_k: int = 5):
    """
    Confusion matrix for the FULL RAG chatbot.
    - y_true = 1 if question has relevant URLs (answerable)
    - y_pred = 1 if chatbot returns at least one relevant URL
    """
    y_true = []
    y_pred = []

    for case in dataset:
        q = case["question"]
        relevant_urls = case["relevant_urls"]

        # Gold label
        gold_label = 1 if len(relevant_urls) > 0 else 0
        y_true.append(gold_label)

        # Run actual chatbot
        answer, urls, retrieved = generate_answer_and_sources(q, top_k=top_k)

        # Predict label
        if urls:
            # chatbot considers it "answerable" if it returned correct URL
            pred_label = 1 if any(any(rel in u for rel in relevant_urls) for u in urls) else 0
        else:
            pred_label = 0

        y_pred.append(pred_label)

    # Make confusion matrix with labels=[1,0] order
    cm = confusion_matrix(y_true, y_pred, labels=[1,0])

    TP, FN = cm[0]
    FP, TN = cm[1]

    print("=== Confusion Matrix (labels=[1,0]) ===")
    print(cm)
    print("\nTP:", TP, " FN:", FN, " FP:", FP, " TN:", TN)

    print("\nDerived metrics:")
    precision = TP / (TP + FP) if (TP + FP) else 0
    recall    = TP / (TP + FN) if (TP + FN) else 0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    print(f"Precision: {precision:.2f}")
    print(f"Recall:    {recall:.2f}")
    print(f"F1 score:  {f1:.2f}")

In [32]:
evaluate_chatbot_confusion_matrix(labeled_chatbot_data, top_k=5)

=== Confusion Matrix (labels=[1,0]) ===
[[1 7]
 [0 2]]

TP: 1  FN: 7  FP: 0  TN: 2

Derived metrics:
Precision: 1.00
Recall:    0.12
F1 score:  0.22
